In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import os.path
from funcs import read_config, get_groups, pull_clean, get_kpi2_respuestas, get_kpi3_retencion, get_kpi4_influencia

In [2]:
business_yelpp = pd.read_parquet('../../datasets/processed/bd/6_business_yelp.parquet.gz')
business_yelpp

,business_id,name,latitude,longitude,avg_stars,state_id
0,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,27.955269,-82.456320,4.09,1
1,0bPLkL0QhhPO5kt1_EXmNQ,Zios Italian Market,27.916116,-82.760461,4.45,1
2,uI9XODGY_2_ieTE6xJ0myw,Roman Forum,28.046203,-82.505053,4.00,1
3,JgpnXv_0XhV3SfbfB50nxw,Joes Pizza,27.960514,-82.506127,4.17,1
4,pJfh3Ct8iL58NZa8ta-a5w,Top Shelf Sports Lounge,28.196252,-82.380615,4.51,1
...,...,...,...,...,...,...
12912,Df-DdXqssj7ZeD8ypAF7Og,Crux Cafe,39.894469,-75.071001,4.00,4
12913,qjtELTt9fdIwoi_xGNN21g,Chipotle Mexican Grill,39.943481,-74.964743,2.43,4
12914,BzPTw9daJW8ToqTc7QiHFw,Taco Bell,40.210771,-74.756588,2.67,4
12915,UBQAksw81m0sMrAd8g-ECg,El Zarape Restaurant,39.718717,-74.971143,4.00,4


In [3]:
#Data Pull and Functions
data_frames = pull_clean() 
state = data_frames.get('1_states.parquet')
categories = data_frames.get('2_categories.parquet')
# user_yelp = data_frames.get('3_user_yelp.parquet')
user_google = data_frames.get('4_user_google.parquet')
business_google = data_frames.get('5_business_google.parquet')
# business_yelp = data_frames.get('6_business_yelp.parquet')
categories_google = data_frames.get('7_categories_google.parquet')
# categories_yelp = data_frames.get('8_categories_yelp.parquet')
reviews_google = data_frames.get('9_reviews_google.parquet')
# reviews_yelp = data_frames.get('10_reviews_yelp.parquet')

1_states.parquet: (4, 2)
2_categories.parquet: (1297, 2)
4_user_google.parquet: (1360619, 5)
5_business_google.parquet: (24755, 6)
6_business_yelp.parquet: (12917, 6)
7_categories_google.parquet: (70307, 2)
9_reviews_google.parquet: (2430142, 8)


In [4]:
groups = get_groups(business_google)

In [5]:
categories

,categories_id,name
0,0,restaurant
1,1,sandwich shop
2,2,caterer
3,3,fast food restaurant
4,4,takeout restaurant
...,...,...
1292,1292,botanical gardens
1293,1293,newspapers & magazines
1294,1294,parking
1295,1295,georgian


In [6]:
groups.head(1)

,gmap_id,name,latitude,longitude,avg_stars,state_id,group
0,0x88d9ba5d65937567:0xbc27649cf513cc89,Bachata Rosa,25.848173,-80.299773,3.6,1,general


In [7]:
business_google.head(1)

,gmap_id,name,latitude,longitude,avg_stars,state_id,group
0,0x88d9ba5d65937567:0xbc27649cf513cc89,Bachata Rosa,25.848173,-80.299773,3.6,1,general


In [29]:
# Fusionar los dataframes en 'gmap_id'
# merged_df = pd.merge(reviews_google, groups, on='gmap_id', how='inner')

# Fusionar con el dataframe de estados para obtener el nombre del estado
merged_df = pd.merge(groups, state, on='state_id', how='inner')

# Fusionar con el dataframe de categorías para obtener el nombre de la categoría
categories_df = pd.merge(categories, categories_google, on='categories_id', how='inner')

# Fusionar con el dataframe de categorías para obtener el nombre de la categoría
merged_df = pd.merge(merged_df, categories_df, on='gmap_id', how='inner')

# Renombrar columnas
merged_df = merged_df.rename(columns={'name_x': 'name', 'name_y': 'category'})

# Borrar columnas residuales
merged_df.drop(columns=['categories_id', 'state_id'],inplace=True)


# # Agrupar por 'gmap_id' y crear una lista de categorías para cada grupo
merged_df = merged_df.groupby('gmap_id').agg({
    'name': 'first',
    'latitude': 'first',
    'longitude': 'first',
    'avg_stars': 'first',
    'state': 'first',
    'group': 'first',
    'category': list
}).reset_index()

# Renombrar la


In [30]:
merged_df

,gmap_id,name,latitude,longitude,avg_stars,state,group,category
0,0x14589e44e1543481:0x488689a13c898c2d,Horizons,30.573791,-81.452990,4.3,Florida,general,"[restaurant, american restaurant]"
1,0x146d33172482a8d5:0xfbb922c5d704e3a5,Mama Mias Pizza,26.157009,-81.796471,3.8,Florida,general,"[pizza restaurant, italian restaurant]"
2,0x14e19dbfedc0c44b:0xe507b98095e6f001,Jeffs Inviting Food and Spirits,33.819972,-117.836713,4.7,California,general,[restaurant]
3,0x14e1b0df6d819dab:0x928b6349b7fbe2f4,Tacos MX,34.045378,-118.253997,4.4,California,general,[mexican restaurant]
4,0x14e1b41c6d02b80f:0xb4ee89d0dd243267,Noodle King,34.080137,-118.110491,3.9,California,general,"[chinese restaurant, taiwanese restaurant]"
...,...,...,...,...,...,...,...,...
24750,0x89c7466629234c93:0x656fa182188b5043,The Landing Restaurant,39.291721,-75.197066,4.1,New Jersey,general,[american restaurant]
24751,0x89c74b5ce1490bcd:0xc9ff3a77c513f29f,Georges Pizzeria,39.257736,-74.987754,4.2,New Jersey,general,"[pizza restaurant, delivery restaurant, italia..."
24752,0x89c74d095a1a9c53:0xeb87fb0cfe6daf80,The Oyster Cracker Cafe,39.233010,-75.032992,4.1,New Jersey,general,[restaurant]
24753,0x89dd13d7011a7cb7:0x30f1c2f732be1d44,TGI Fridays,40.340751,-74.306614,3.5,New Jersey,general,"[restaurant, american restaurant, bar & grill]"


In [9]:
merged_df.drop_duplicates()

,categories_id,name,gmap_id
0,0,restaurant,0x88d9ba5d65937567:0xbc27649cf513cc89
1,0,restaurant,0x88d900575f0dd065:0x9b3638d2a80be4d
2,0,restaurant,0x88d9b719170a9f61:0x30d454a980f76ad3
3,0,restaurant,0x88e76652cd84272f:0x548abb9935d912ff
4,0,restaurant,0x88d9b395c6179c89:0x914b0aafb453b3b5
...,...,...,...
70302,794,condominium complex,0x89c0bc82a804da0b:0xe9466fa5a11862a3
70303,795,equipment supplier,0x89c1846c5f430859:0x69ba082723f8b32d
70304,796,public bathroom,0x89c1026295b55cb7:0x24756dd82d799a5d
70305,797,garden,0x89c3ccd5de4abc17:0x93a5267ffe05b932


In [11]:
merged_df

,categories_id,name,gmap_id
0,0,restaurant,0x88d9ba5d65937567:0xbc27649cf513cc89
1,0,restaurant,0x88d900575f0dd065:0x9b3638d2a80be4d
2,0,restaurant,0x88d9b719170a9f61:0x30d454a980f76ad3
3,0,restaurant,0x88e76652cd84272f:0x548abb9935d912ff
4,0,restaurant,0x88d9b395c6179c89:0x914b0aafb453b3b5
...,...,...,...
70302,794,condominium complex,0x89c0bc82a804da0b:0xe9466fa5a11862a3
70303,795,equipment supplier,0x89c1846c5f430859:0x69ba082723f8b32d
70304,796,public bathroom,0x89c1026295b55cb7:0x24756dd82d799a5d
70305,797,garden,0x89c3ccd5de4abc17:0x93a5267ffe05b932
